<a href="https://colab.research.google.com/github/hamk3010/DeepLearning/blob/master/Linear%20Regression%20with%20TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

These examples were found in [Hands-On Machine Learning with Scikit-Learn & Tensorflow](http://index-of.es/Varios-2/Hands%20on%20Machine%20Learning%20with%20Scikit%20Learn%20and%20Tensorflow.pdf)

In [0]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import fetch_california_housing

# Fetch the housing dataset
housing = fetch_california_housing()
m,n = housing.data.shape

# Add a bias input feature (x0 = 1) to all training instances
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]

In [14]:
 print(tf.__version__)

1.13.0-rc0


In [0]:
#  Construction phase
# Create two TensorFlow constant nodes, X and y to hold this data and the targets
X = tf.constant(housing_data_plus_bias, dtype= tf.float32, name = "X")
y = tf.constant(housing.target.reshape(-1,1), dtype= tf.float32, name = "y")
XT = tf.transpose(X)
theta = tf.matmul( tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

In [0]:
# Execution phase

with tf.Session() as sess:
    theta_value = theta.eval()

In [5]:
print(theta_value)

[[-3.71037292e+01]
 [ 4.36282694e-01]
 [ 9.40542948e-03]
 [-1.06901854e-01]
 [ 6.43611908e-01]
 [-4.06625077e-06]
 [-3.78273334e-03]
 [-4.23094332e-01]
 [-4.36462164e-01]]


In [0]:
# Lets normalize the input feature vectors first
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(housing_data_plus_bias)
scaled_housing_data_plus_bias = scaler.transform(housing_data_plus_bias)

# Implementing Gradient Descent
We will implement gradient descent by first manually compute the gradients, then we will use *autodiff* to compute the gradients automatically. Additioanlly, we will also use optimizers that come with TensorFlow.

In [12]:
#  Construction phase
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype= tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype= tf.float32, name = "Y")

# Create a tensor containing random values 
theta = tf.Variable(tf.random_uniform([ n + 1 , 1 ], -1.0, 1.0), name = "theta")
y_pred = tf.matmul(X, theta, name = "predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = "mse")
# Manually compute gradients
gradients = 2/m * tf.matmul(tf.transpose(X), error)

# assign() function creates a new node that will assign a new value to a variable
#  theta[n] = theta for the next step
#  theta = theta in this step
#  
# Over here it is impplementing Batch Gradient Descent step
#  theta[n] = theta - learning_rate * gradients
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()


Instructions for updating:
Colocations handled automatically by placer.


In [13]:
with tf.Session() as sess:
  sess.run(init)
  
  for epoch in range(n_epochs):
    if(epoch%100 == 0):
      print("Epoch", epoch, "MSE = ", mse.eval() )
      sess.run(training_op)
  best_theta = theta.eval()

Epoch 0 MSE =  9.255035
Epoch 100 MSE =  9.054788
Epoch 200 MSE =  8.864392
Epoch 300 MSE =  8.683327
Epoch 400 MSE =  8.511095
Epoch 500 MSE =  8.347228
Epoch 600 MSE =  8.191288
Epoch 700 MSE =  8.04286
Epoch 800 MSE =  7.901554
Epoch 900 MSE =  7.767001
